In [1]:
import json
import pandas as pd
import pickle
import os
import shutil

In [31]:
'config_msr_action3d_temporal_1024_16_128_128_0.001'.split('_')

['config', 'msr', 'action3d', 'temporal', '1024', '16', '128', '128', '0.001']

In [33]:
hold_out_score = json.load(open(r'holdout_scores.json'))
df_hold_out_score = pd.DataFrame(hold_out_score)
df_hold_out_score

,model,accuracy
0,config_msr_action3d_temporal_1024_16_128_128_0...,4.807692
1,config_msr_action3d_temporal_1024_16_128_128_0...,6.129808
2,config_msr_action3d_temporal_1024_16_128_128_0.05,4.807692
3,config_msr_action3d_temporal_1024_16_128_32_0.001,6.370192
4,config_msr_action3d_temporal_1024_16_128_128_0...,4.447115
...,...,...
64,config_msr_action3d_temporal_2048_16_128_64_0.001,6.730769
65,config_msr_action3d_temporal_2048_16_128_64_0.025,4.807692
66,config_msr_action3d_temporal_2048_16_128_64_0.05,4.807692
67,config_msr_action3d_temporal_2048_16_32_128_0.001,5.288462


In [34]:
df_hold_out_score['dim_hidden'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[4])
df_hold_out_score['num_heads'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[5])
df_hold_out_score['num_inds'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[6])
df_hold_out_score['batch_size'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[7])
df_hold_out_score['lr'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[8])

In [35]:
df_hold_out_score[df_hold_out_score['accuracy'] > 5]

,model,accuracy,dim_hidden,num_heads,num_inds,batch_size,lr
1,config_msr_action3d_temporal_1024_16_128_128_0...,6.129808,1024,16,128,128,0.025
3,config_msr_action3d_temporal_1024_16_128_32_0.001,6.370192,1024,16,128,32,0.001
7,config_msr_action3d_temporal_1024_16_128_32_0.001,6.129808,1024,16,128,32,0.001
10,config_msr_action3d_temporal_1024_16_128_64_0.001,6.610577,1024,16,128,64,0.001
11,config_msr_action3d_temporal_1024_16_128_64_0.025,6.490385,1024,16,128,64,0.025
13,config_msr_action3d_temporal_1024_16_32_128_0.001,6.610577,1024,16,32,128,0.001
16,config_msr_action3d_temporal_1024_16_32_32_0.001,5.168269,1024,16,32,32,0.001
19,config_msr_action3d_temporal_1024_16_32_64_0.001,5.168269,1024,16,32,64,0.001
20,config_msr_action3d_temporal_1024_16_32_64_0.025,5.048077,1024,16,32,64,0.025
25,config_msr_action3d_temporal_1024_16_64_32_0.001,9.615385,1024,16,64,32,0.001


In [36]:
keep_models = df_hold_out_score[df_hold_out_score['accuracy'] > 5]['model'].to_list()
keep_models

['config_msr_action3d_temporal_1024_16_128_128_0.025',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.025',
 'config_msr_action3d_temporal_1024_16_32_128_0.001',
 'config_msr_action3d_temporal_1024_16_32_32_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.025',
 'config_msr_action3d_temporal_1024_16_64_32_0.001',
 'config_msr_action3d_temporal_1024_16_64_64_0.001',
 'config_msr_action3d_temporal_1024_4_128_32_0.025',
 'config_msr_action3d_temporal_1024_4_128_64_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.025',
 'config_msr_action3d_temporal_1024_4_32_64_0.025',
 'config_msr_action3d_temporal_1024_4_64_128_0.001',
 'config_msr_action3d_temporal_1024_4_64_64_0.001',
 'config_msr_action3d_temporal_2048_16_128_

In [37]:
path = r'log'

model_list_temp = [x for x in os.listdir(path) if 'msr_action3d_temporal' in x]
model_list_temp

['config_msr_action3d_temporal_1024_16_128_128_0.025',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.025',
 'config_msr_action3d_temporal_1024_16_32_128_0.001',
 'config_msr_action3d_temporal_1024_16_32_32_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.025',
 'config_msr_action3d_temporal_1024_16_64_32_0.001',
 'config_msr_action3d_temporal_1024_16_64_64_0.001',
 'config_msr_action3d_temporal_1024_4_128_32_0.025',
 'config_msr_action3d_temporal_1024_4_128_64_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.025',
 'config_msr_action3d_temporal_1024_4_32_64_0.025',
 'config_msr_action3d_temporal_1024_4_64_128_0.001',
 'config_msr_action3d_temporal_1024_4_64_64_0.001',
 'config_msr_action3d_temporal_2048_16_128_128_0.001',
 'config_msr_action3d_temporal_2048_16_128

In [38]:
for model in model_list_temp:
    if model not in keep_models:
        print(model)
        #shutil.rmtree(os.path.join(path, model))